# 네이버 영화 리뷰 데이터를 크롤링하여 NLP를 최대한 할 수 있는 영역으로 자유롭게 분석 부탁드립니다.

### 배경 
1. 영화는 하나의 미디어 상품이고 그 상품을 구매하는 소비자는 시각적인 즐거움뿐만 아니라 감정을 극대화 해주는 즐거움을 산다.
2. 좋은 영화라고 말하는 것은 대부분 자신이 영화에 동화되어 감정이 잘 움직인 영화일 것이다.(스토리, 영상미, 배우 등을 통해서)
3. 역으로 나쁜 영화라는 것은 영화에 동화되지 못하여 감정이 잘 움직이지 않은 영화일 것이다.
4. 소비자는 리뷰를 쓸 때 귀찮음을 이겨내고 쓰기 때문에 리뷰를 남긴다는 것은 커다란 감정을 받았을 가능성이 크다.
5. 그래서 영화 리뷰는 어떤 단어들로 썼는지 군집화를 해서 어떤 감정들을 이야기 하고 싶었는지 소비자의 생각을 알아보자.


### 분석 절차
1. 데이터 범위 : 네이버 영화 리뷰의 10000까지의 데이터 중 3000개의 리뷰
2. 데이터 가공 : 
  - Beautiful soup를 사용해서 네이버의 영화 리뷰 정보 스크래핑 하였다.
  - 스크래핑한 리뷰를 군집화 하기 위해 TfidfVectorizer 알고리즘을 사용해서 단어를 숫자로 변환하는 벡터화를 진행하였다.
  - ngram_range를 1,2로 하여 1개의 단어만 나오도록 하였다.
  - 벡터화 중 파라미터인 tokenizer를 한글 형태소 엔진인 KoNLPy안의 twitter를 사용하여 단어를 형태소화 진행
  - KMeans 클러스터링 알고리즘을 사용하여 사용하여 벡터화된 단어를 군집화(군집화의 개수를 조정해봐야 함)
  - 군집별로 가장많이 나온 단어 상위 10개의 단어를 추출해 그 군집이 어떤 단어로 뭉쳐져 있는지 파악
3. 분석
  - KMeans 클러스터링은 군집이 어떻게 저장되는지를 알지 못하므로 여러개의 군집으로 나눠보고  빈도수가 많은 상위 단어를 보면서 군집을 유추

### 결과
1. 기본적으로 TfidfVectorizer는 한글의 불용어를 처리하지를 못하기 때문에 불용어가 많아 분석하기 쉽지 않다.
2. 군집을 줄여보고 필요한 추출 단어도 늘려봤지만 감정을 나타내는 단어보단 배우, 스토리등을 설명하는 단어가 압도적으로 많이 들어가 있었다.
3. 감정을 분석하기 위해선 역으로 감정적인 단어의 단어집을 만들고 그 단어가 몇 번나오는지를 벡터화 하여 군집을 진행하여야 감정분석이 가능할 것이다.


In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
from konlpy.tag import Twitter
import numpy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
temp = list(range(1, 10000)) # 만개
rand_order = random.sample(temp, k=3000)

In [3]:
review_list = []
for i in rand_order:
    base_url = 'https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid={}&code=36944&order=#tab'.format(i)
    response = requests.get(base_url)
    soup = bs(response.text, 'html.parser')
    content = soup.select("#content > div.article > div.obj_section.noline.center_obj > div.review > div.user_tx_area")
    
    if content == []:
        continue
    else:
        content_text = content[0].get_text(strip=True).replace('\n', ' ').replace('\r', ' ')
        review_list.append(content_text)
    
review_df = pd.DataFrame({'text':review_list})

In [4]:
review_df.head()

,text
0,다운받아서 오늘에서야 겨우 보게 됐는데 역쉬 생각했던것만큼 재미는 있네여 ...
1,정말 오랫만에 실감나게 재미있는 SF영화였습니다. 돌연변이 인간이 생길법한 시대 ...
2,너무나 이쁘고 아름다은 사랑이지만 너무나두 말이 안&#46080;다~~ 그렇게 까지...
3,어떤 영화를 잼있게 보셨는지 심히 의심스럽네요.. 코미디 수준이 좀 떨어지시는...
4,전 개인적으로 제발 대박좀 났으면 하는 바램이에요. 그도 그럴것이 우리나라에선 소위...


In [5]:
twitter = Twitter()
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [123]:
# TfidfVectorizer에는 stop_words가 english 밖에 없기 때문에 내가 stop words를 넣어준다.
stop_word_list = ['\n', '\r', '>', '이', '영화', '의', '가', '에', '을', '를', '..', '.', '?', '.', '의',
                  '을', '이', ',', '를', '영화', '에', '들', '가','...', '이', '..', '영화', '의', '을', '에',
                  '가', '....', '.','`', '은', '는', '그', '것', '한', '도', '으로', '에서', '로',
                 '정말', '은', '너무', '다', '한', '는', '로', '과', '입니다','~', '넘', '정말', '도', '는', '여',
                  '^^', '너무', '은', '꼭', '2', '편', '1', '3', '(', ')','와', '더','만', '잘','!', '요','저', '말',
                  '제', '하는', '하고', '만', '생각', '안', '보고', '내용', '사람', '좀', '장면', '못', '고', '분''"',
                  '생각', '이다', '적', '사람', '장면', '나', '때', '인', '감독','적', '인', '적 인', '생각', '감동',
                  '감독', '개인', '개인 적', '장면', '하지만', '그리고', '특히', '볼', '본','수', '할', '우리', '있는',
                  '에게', '한다', '네', '진짜','-', '분', '이런', '합니다', '정도', '그리고', '"', '라는', '수',
                  '내', '속', '인간', '부분', '서', '된', '두', '전', '왜', '지', '한번',
                  '대한', '같은', '그런', '많은','참',';','이영화','없는', '느낌', '보다', '면', '주인공',
                  '자신', '그런', '일', '까지', '라고', '모습', '하나', '때문', '주인공', '있다',
                  '마지막', '많이','게', '마지막', '해', '사실','점','보면', '였습니다' '봤는데', '라',
                  '해서','님','이라는', '에는', '중', '성', '였다','역시','랑', '근데', '해서', '시간',
                  '였습니다', '같다', '인지', '없이', '였습니다', '극장', '같다', '아주', '이나', '또', '알', '하게',
                  '스토리', '이해', '이야기', '글', '작품', '거', '뭐', '돈', '다른', '보는', '아',
                  '처음', '관객', '기대', '부터', '조금', '듯', '다시','처럼', '가슴', '여자', '위해', '작품',
                  '가장', '좋은', '다른', '연기', '난', '함께', '모든','이란', '연기', '배우', '좋은', '작품',
                  '같습니다', '봤습니다', '조금', '난', '기', '가장', '다시', '보는',
                  '다른', '가슴', '처음']

In [124]:
# tfidf를 사용하여 단어를 토큰화 한다.
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1, 2), stop_words=stop_word_list, min_df=0.05, max_df=0.85)
feature_vect = tfidf_vect.fit_transform(review_df['text'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['봤는데'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [125]:
km_cluster = KMeans(n_clusters=5, max_iter=1000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

In [126]:
# 군집화된 그룹별로 데이터 확인
review_df['cluster_label'] = cluster_label
review_df.head()

,text,cluster_label
0,다운받아서 오늘에서야 겨우 보게 됐는데 역쉬 생각했던것만큼 재미는 있네여 ...,1
1,정말 오랫만에 실감나게 재미있는 SF영화였습니다. 돌연변이 인간이 생길법한 시대 ...,2
2,너무나 이쁘고 아름다은 사랑이지만 너무나두 말이 안&#46080;다~~ 그렇게 까지...,0
3,어떤 영화를 잼있게 보셨는지 심히 의심스럽네요.. 코미디 수준이 좀 떨어지시는...,1
4,전 개인적으로 제발 대박좀 났으면 하는 바램이에요. 그도 그럴것이 우리나라에선 소위...,3


In [127]:
display(review_df[review_df['cluster_label'] == 0])
display(review_df[review_df['cluster_label'] == 1])
display(review_df[review_df['cluster_label'] == 2])
display(review_df[review_df['cluster_label'] == 3])
display(review_df[review_df['cluster_label'] == 4])

,text,cluster_label
2,너무나 이쁘고 아름다은 사랑이지만 너무나두 말이 안&#46080;다~~ 그렇게 까지...,0
14,"사랑은 이렇게 복잡하고,힘들고,어려운것인거다. 쉽게 다가온 사랑은 그만큼 쉽게 떠...",0
35,내가 본 중국영화중에 가장 재밌고 감동적이엇던 영화다. 3번 넘게 봤던가........,0
51,줄거리 푸른 하늘을 약속한 남자 VS 금지된 사랑을 시작한 여자 희망의 이름...,0
59,이제껏 알아왔던 신현준의 이미지와는 전혀다른 모습을 봤다. 은행나무침대나 그외 다른...,0
...,...,...
2757,영화 방금 봤는데요.. 잘봤습니다.. 현대 인스턴트적사랑시대에 조금이나마 순수한 ...,0
2771,멜 깁슨 하나로 승부를 거는 영화같아요.확실히 멋잇기는 하죠.>.< 정말 제목 그대...,0
2797,참으로..이영화를 오락 영화라고 평가할수 밖에 없다. 액션이라 하기에는 액션이 밋밋...,0
2807,가슴아픈 사랑 이야기 였습니다. 저는 이영화를 3번이나 보았지만 보면 볼수록 감동적...,0


,text,cluster_label
0,다운받아서 오늘에서야 겨우 보게 됐는데 역쉬 생각했던것만큼 재미는 있네여 ...,1
3,어떤 영화를 잼있게 보셨는지 심히 의심스럽네요.. 코미디 수준이 좀 떨어지시는...,1
5,넘 웃겨서 할말을 잃는 영화... 기존의 스파이영화와 차별화된 코믹스러우면서도 ...,1
6,"원래 해리포터 팬이라서.. 해리포터가 최고인줄 알고 반지의 제왕 1,2편 개봉...",1
7,로만 폴라스키 감독은 이 영화를 위해 태어났다... 말이 필요없다.....,1
...,...,...
2862,"이영화는 무슨내용일까, 무엇을 말하려고 했을까를 논하는것이 어리석다고 생각한다. ...",1
2863,나 꼭 보고싶은데 관람이 되질않아 너무 속상해 하지만 빨리 나이먹어서 꼭 볼꺼...,1
2864,저는 국화꽃 향기를 책으로 먼저 보고 영화로도 봤는데요.. 책이 더 나...,1
2865,정말.. 실망이 이만저만이 아니였습니다 역시 반지의 제왕을 봤어야 되는건데;; ...,1


,text,cluster_label
1,정말 오랫만에 실감나게 재미있는 SF영화였습니다. 돌연변이 인간이 생길법한 시대 ...,2
50,"로보캅, 토탈리콜등을 만든 폴버호벤감독의 영화다. 출연진으로는 주인공 캐스퍼 반디엔...",2
62,어제 여친이 헐크보러가자 해서 보러갔다가 갑자기 이거보자길래 재미있을까??? 1...,2
79,"장르를 따지신다면 액션, 멜로, 추리, 그리고 환각의 세계를 떠나는 판타지 모두를 ...",2
122,시사회에 당첨이 되어 칭구랑 스카라극장가서 영화를 보게 되었답니다..간만에 영화를 ...,2
...,...,...
2751,많은 평들이 있는데 정말 다 미국 현지에서 보고 직접 평올려주신걸까요? 물론 직...,2
2790,기대안하고 봤는데.. 평 그대로 잼있네여... 매트릭스와 액션부분을 비교...,2
2809,"이상하게 우리나라에서 개봉을 안하네여, 흥행 수지가 안 맞아서 그런가? 제가 그동...",2
2822,정말 재밌었다 영화보는내내 눈을 즐겁게해주는 그런영화 였다 배우들의 연기...,2


,text,cluster_label
4,전 개인적으로 제발 대박좀 났으면 하는 바램이에요. 그도 그럴것이 우리나라에선 소위...,3
13,제가 이해한 게 맞는지 모르겠지만 1편에서의 매트릭스가 가장 안에 있는 매트릭스이고...,3
18,"사토라레, 초능력도 때로는 짐만 된다? (Satorare, 2001) 감독 ...",3
19,전 개인적으로 한국영화를 좋아 합니다.. 외화는 그 원음을 잘 전달 하지 못하...,3
22,아직 나올 시기는 멀었죠. 하지만 이 영화 홍보가 미숙해서인지 한국에서는 모...,3
...,...,...
2844,요즘 바빠서 영화에 신경쓸 시간이 없었는데 우연하게 이 영화를 보게 되었다. (=>...,3
2850,한가지~! 이유로 뭉쳐진 아니 버려진 그들..어떻게든.. 엮히지 않던 그들을 ...,3
2852,"어제(2003.11.7)저녁에 신랑이랑 영화를 보았는데, 줄거리도 있고, 화면 화면...",3
2853,이 영화의 키포인트이자 .테마는 뭔가? 다른 말 할거 없다 전편의 조폭 마누라...,3


,text,cluster_label
25,"솔직히 그렇게 재밌지는 않던데요.. 사실 기대도 별로 안하고 그냥 봤었거든요,...",4
48,첨에 이거 예고를 보고 엄청 기대했었거든요 ^^ 근데 본사람들마다 재미없다고 그러...,4
54,앙숙인 두 남녀가 벌이는 한바탕 소동이 대규모로 이루어지는 영화. 급박한 혁명 속...,4
65,이걸 본 사람이 아무도 없네요 주위에 2명이 이걸 봤는데..저에게 추천을 하지 않...,4
123,내가 본영화중 사상최악의 영화 이영화는 12세이상 상영금지 시켜야 합니다.조용하다가...,4
...,...,...
2794,"1편과 2편을 본 후 저의 솔직한 느낌은 비쥬얼, 특수효과 외엔 건질 게 없는 평범...",4
2848,가장 무서웠던 장면은 기자가 여학생을 인터뷰한 장면을 계속 리플레이하며 보다가 그티...,4
2851,셈세하다 하나하나 꼼꼼하고 환상적이다.....간달프가 마법잘안쓰는게 좀답답했다......,4
2859,반담영화 많이 본건 아니지만 몇몇 영화는 잘봤는데 대부분 그냥 그런 액션영화였...,4


In [128]:
# 군집별 핵심 단어 추출하기
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers:', cluster_centers.shape)
print(cluster_centers)

cluster_centers: (5, 128)
[[0.01730115 0.02226009 0.01222578 0.01964343 0.01583708 0.00996466
  0.01709611 0.01427353 0.01645265 0.01810833 0.00860999 0.0239496
  0.00949456 0.00845869 0.02432593 0.02976656 0.01278531 0.02390737
  0.02969787 0.01806217 0.01938206 0.01110592 0.04228212 0.01218601
  0.03794188 0.00866745 0.01354317 0.01626821 0.03162924 0.02355025
  0.0185286  0.02543888 0.0121304  0.03009132 0.01790882 0.01319769
  0.02207964 0.02721173 0.01737147 0.05653711 0.01490137 0.01424252
  0.02842745 0.00927137 0.02161504 0.02488752 0.01675344 0.01371891
  0.01871411 0.02084573 0.01323366 0.02449791 0.0094099  0.01716437
  0.0277523  0.00748031 0.0098967  0.01338715 0.02389191 0.52047409
  0.01067313 0.01301307 0.02368578 0.012735   0.01231098 0.02159295
  0.01976774 0.01802522 0.01692042 0.01664875 0.01446981 0.01184593
  0.01915713 0.01459612 0.01774042 0.01255326 0.02021853 0.01405626
  0.02189568 0.00838928 0.03199441 0.0148345  0.01507128 0.01058622
  0.01523317 0.01888136

In [129]:
 # 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        
    return cluster_details

In [130]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('[cluster {}]'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('==================================================\n')

In [131]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=review_df,\
                                  feature_names=feature_names, clusters_num=5, top_n_features=20)
print_cluster_details(cluster_details)

[cluster 0]
Top features: ['사랑', '마음', '남자', '눈물', '친구', '하지', '추천', '엔', '대해', '했다', '재미', '된다', '그러나', '기억', '모두', '지금', '보면서', '또한', '했던', '되는']

[cluster 1]
Top features: ['친구', '재미', '추천', '봤는데', '엔', '보지', '비디오', '그래도', '영', '끝', '니', '구', '반전', '솔직히', '.....', '했는데', '최고', '실망', '보기', '같이']

[cluster 2]
Top features: ['액션', '에서는', '봤는데', '추천', '실망', '재미', '별로', '인데', '그냥', '하', '하지', '큰', '그래도', '전혀', '엔', '영', '보기', '약간', '솔직히', '식']

[cluster 3]
Top features: ['우리나라', '모두', '그것', '한국', '위', '가지', '또한', '죠', '했다', '지금', '물론', '하지', '이유', '되는', '에서는', '그렇게', '이라고', '이고', '등', '현실']

[cluster 4]
Top features: ['그냥', '재미', '솔직히', '친구', '죠', '엔', '별로', '영', '그런데', '그렇게', '대로', '뿐', '웃음', '하', '보다는', '날', '건', '실망', '약간', '하지']

